In [ ]:
import pickle

with open('message.pickle', 'wb') as handle:
    pickle.dump(message, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('message.pickle', 'rb') as handle:
    b = pickle.load(handle)

In [20]:
import sqlalchemy

from sqlalchemy import create_engine
from sqlalchemy.orm import declarative_base

engine = create_engine('sqlite:///bot-converter.db', echo=True)
Base = declarative_base()

In [21]:
from sqlalchemy.orm import close_all_sessions

close_all_sessions()

Exception during reset or similar
Traceback (most recent call last):
  File "/root/workspace/bot-converter/venv/lib/python3.8/site-packages/sqlalchemy/pool/base.py", line 739, in _finalize_fairy
    fairy._reset(pool)
  File "/root/workspace/bot-converter/venv/lib/python3.8/site-packages/sqlalchemy/pool/base.py", line 988, in _reset
    pool._dialect.do_rollback(self)
  File "/root/workspace/bot-converter/venv/lib/python3.8/site-packages/sqlalchemy/engine/default.py", line 682, in do_rollback
    dbapi_connection.rollback()
sqlite3.ProgrammingError: SQLite objects created in a thread can only be used in that same thread. The object was created in thread id 140479429592832 and this is thread id 140480232879936.
Exception closing connection <sqlite3.Connection object at 0x7fc4045c14e0>
Traceback (most recent call last):
  File "/root/workspace/bot-converter/venv/lib/python3.8/site-packages/sqlalchemy/pool/base.py", line 739, in _finalize_fairy
    fairy._reset(pool)
  File "/root/workspa

2022-01-20 17:41:55,129 INFO sqlalchemy.engine.Engine ROLLBACK


ProgrammingError: (sqlite3.ProgrammingError) SQLite objects created in a thread can only be used in that same thread. The object was created in thread id 140479412807424 and this is thread id 140480232879936.
(Background on this error at: https://sqlalche.me/e/14/f405)

In [63]:
from sqlalchemy import Column, Integer, String
class User(Base):
    __tablename__ = 'users'

    id = Column(Integer, primary_key=True)
    chat_id = Column(Integer)
    filename = Column(String)

    def __repr__(self):
        return "<User(chat_id='{}', filename='{}')>".format(self.chat_id, self.filename)

In [64]:
Base.metadata.create_all(engine)

2022-01-20 16:35:10,260 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-01-20 16:35:10,262 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("users")
2022-01-20 16:35:10,263 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-01-20 16:35:10,268 INFO sqlalchemy.engine.Engine COMMIT


In [65]:
ed_user = User(chat_id=0, filename='test.mov')

In [66]:
from sqlalchemy.orm import sessionmaker
Session = sessionmaker(bind=engine)
session = Session()

In [67]:
session.add(ed_user)

In [68]:
our_user = session.query(User).filter_by(chat_id=0).first() 
print(our_user)

2022-01-20 16:35:13,197 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-01-20 16:35:13,204 INFO sqlalchemy.engine.Engine INSERT INTO users (chat_id, filename) VALUES (?, ?)
2022-01-20 16:35:13,205 INFO sqlalchemy.engine.Engine [generated in 0.00164s] (0, 'test.mov')
2022-01-20 16:35:13,209 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.chat_id AS users_chat_id, users.filename AS users_filename 
FROM users 
WHERE users.chat_id = ?
 LIMIT ? OFFSET ?
2022-01-20 16:35:13,212 INFO sqlalchemy.engine.Engine [generated in 0.00370s] (0, 1, 0)
<User(chat_id='0', filename='test.mov')>


In [69]:
str(our_user)

"<User(chat_id='0', filename='test.mov')>"

In [70]:
session.commit()

2022-01-20 16:35:18,898 INFO sqlalchemy.engine.Engine COMMIT


In [ ]:
from sqlalchemy import Column, Integer, String
from sqlalchemy.orm import declarative_base
from sqlalchemy import create_engine

engine = create_engine('sqlite:///bot-converter.db', echo=True)
Base = declarative_base()


class User(Base):
    __tablename__ = 'users'

    id = Column(Integer, primary_key=True)
    chat_id = Column(Integer)
    filename = Column(String)

    def __repr__(self):
        return "<User(chat_id='{}', filename='{}')>".format(self.chat_id, self.filename)

from sqlalchemy import create_engine
engine = create_engine('sqlite:///bot-converter.db', echo=True)
from sqlalchemy.orm import sessionmaker
Session = sessionmaker(bind=engine)


import telebot
from telebot import types

bot = telebot.TeleBot("5069674898:AAGdHYM4cn-ZqQqbhR2vke34ke8qMos1lTY", parse_mode=None)

WELCOME_MSG = "👋 Howdy, how are you doing? \
I can convert video to the following formats: *.avi, *.mov, *.mp4! and convert to *.avi, *.mov, *.mp4, *.gif!\
Just upload a short video for me, and then choose file type you want."

@bot.message_handler(commands=['start', 'help'])
def send_welcome(message):
	bot.reply_to(message, WELCOME_MSG)

@bot.message_handler(content_types=['video', 'video_note', 'document'])
def handle_docs_video(message):
    print(message.content_type)
    # Get file
    if message.content_type == 'video_note':
        file_info = bot.get_file(message.video_note.file_id)
    elif message.content_type == 'video':
        file_info = bot.get_file(message.video.file_id)
    elif message.content_type == 'document':
        file_info = bot.get_file(message.document.file_id)
    
    ## Check if filetype is supported 
    file_type = file_info.file_path.split('.')[-1].lower()
    if (file_type in ['mov', 'mp4', 'avi']):
        bot.reply_to(message, 'Detected file type: "{}". Congrats! It is supported!'.format(file_type))
    else:
        bot.reply_to(message, 'Detected file type: "{}". Sorry is not supported!'.format(file_type))
    
    ## Save temporary file
    downloaded_file = bot.download_file(file_info.file_path)
    temp_filename = str(file_info.file_path).split('/')[-1]
    with open(temp_filename,'wb') as new_file:
        new_file.write(downloaded_file)
        
    # Get target filetype
    markup = types.ReplyKeyboardMarkup(row_width=2)
    itembtn1 = types.KeyboardButton('/avi')
    itembtn2 = types.KeyboardButton('/mov')
    itembtn3 = types.KeyboardButton('/mp4')
    itembtn4 = types.KeyboardButton('/gif')
    
    

    markup.add(itembtn1, itembtn2, itembtn3, itembtn4)
    bot.send_message(message.chat.id, "Choose the target file type:", reply_markup=markup)
    
    ed_user = User(chat_id=message.chat.id, filename=temp_filename)
    session = Session()
    session.add(ed_user)
    session.commit()
    session.close()

@bot.message_handler(commands=['avi', 'mov', 'mp4', 'gif'])
def process_file(message):
    print('processing file')
    markup = types.ReplyKeyboardRemove(selective=False)
    target_file_type = message.text[1:]
    bot.send_message(message.chat.id, 'You\'ve chosen .{} format! Processing...'.format(target_file_type), reply_markup=markup)
    
    session = Session()
    our_user = session.query(User).filter_by(chat_id=message.chat.id).order_by(User.id.desc()).first()
    session.close()
    
    temp_filename = our_user.filename
    
    # Convert file
    import ffmpeg
    try:
        stream = ffmpeg.input(temp_filename)
        output_name = '{}.{}'.format(temp_filename.split('.')[0], target_file_type)
        stream = ffmpeg.output(stream, output_name)
        ffmpeg.run(stream)
    except Exception as e:
        print(e)
        bot.send_message(message.chat.id, "Ooops! An error occured! Try again! {}".format(e))
    
    # Send file
    with open(output_name,'rb') as converted_file:
        bot.send_document(message.chat.id, converted_file)
        import os
    os.remove(output_name)

@bot.message_handler(func=lambda message: True)
def echo_all(message):
	bot.reply_to(message, message.text)

bot.infinity_polling()

processing file
2022-01-20 18:10:18,336 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-01-20 18:10:18,341 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.chat_id AS users_chat_id, users.filename AS users_filename 
FROM users 
WHERE users.chat_id = ? ORDER BY users.id DESC
 LIMIT ? OFFSET ?
2022-01-20 18:10:18,344 INFO sqlalchemy.engine.Engine [generated in 0.00330s] (26921314, 1, 0)
2022-01-20 18:10:18,348 INFO sqlalchemy.engine.Engine ROLLBACK
ffmpeg error (see stderr output for detail)
processing file
2022-01-20 18:10:32,958 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-01-20 18:10:32,962 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.chat_id AS users_chat_id, users.filename AS users_filename 
FROM users 
WHERE users.chat_id = ? ORDER BY users.id DESC
 LIMIT ? OFFSET ?
2022-01-20 18:10:32,965 INFO sqlalchemy.engine.Engine [cached since 14.62s ago] (26921314, 1, 0)
2022-01-20 18:10:32,969 INFO sqlalchemy.engine.Engine ROLLBACK


2022-01-20 18:10:33,119 (__init__.py:621 MainThread) ERROR - TeleBot: "Infinity polling exception: [Errno 2] No such file or directory: 'file_61.gif'"


ffmpeg error (see stderr output for detail)


2022-01-20 18:10:33,124 (__init__.py:623 MainThread) ERROR - TeleBot: "Exception traceback:
Traceback (most recent call last):
  File "/root/workspace/bot-converter/venv/lib/python3.8/site-packages/telebot/__init__.py", line 617, in infinity_polling
    self.polling(none_stop=True, timeout=timeout, long_polling_timeout=long_polling_timeout,
  File "/root/workspace/bot-converter/venv/lib/python3.8/site-packages/telebot/__init__.py", line 664, in polling
    self.__threaded_polling(non_stop, interval, timeout, long_polling_timeout, allowed_updates)
  File "/root/workspace/bot-converter/venv/lib/python3.8/site-packages/telebot/__init__.py", line 726, in __threaded_polling
    raise e
  File "/root/workspace/bot-converter/venv/lib/python3.8/site-packages/telebot/__init__.py", line 686, in __threaded_polling
    self.worker_pool.raise_exceptions()
  File "/root/workspace/bot-converter/venv/lib/python3.8/site-packages/telebot/util.py", line 136, in raise_exceptions
    raise self.exception_i

video_note
2022-01-20 18:10:45,794 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-01-20 18:10:45,800 INFO sqlalchemy.engine.Engine INSERT INTO users (chat_id, filename) VALUES (?, ?)
2022-01-20 18:10:45,802 INFO sqlalchemy.engine.Engine [generated in 0.00277s] (26921314, 'file_62.mp4')
2022-01-20 18:10:45,806 INFO sqlalchemy.engine.Engine COMMIT
processing file
2022-01-20 18:10:47,168 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-01-20 18:10:47,173 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.chat_id AS users_chat_id, users.filename AS users_filename 
FROM users 
WHERE users.chat_id = ? ORDER BY users.id DESC
 LIMIT ? OFFSET ?
2022-01-20 18:10:47,175 INFO sqlalchemy.engine.Engine [cached since 28.83s ago] (26921314, 1, 0)
2022-01-20 18:10:47,179 INFO sqlalchemy.engine.Engine ROLLBACK
processing file
2022-01-20 18:10:58,986 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-01-20 18:10:58,991 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, user

In [16]:
b.document.file_name

NameError: name 'b' is not defined